In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from analyzer import *
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [3]:
labels = ["l1", "l2"]
X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=0, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [4]:
analyzer = Analyzer(root_dir="..")

In [75]:
analyzer.model_overview(y_test, y_pred, labels, "1.0", plot_metrics=False, overwrite=True)

In [6]:
analyzer.compare_models("1.0", "1.1", overwrite=True)

# Experementing Below

In [12]:
from typing import Optional
import re

def extract_key(string: str, pattern: str) -> Optional[str]:
    match = re.match(pattern.replace("<<key>>", "(.+)"), string)
    if match: return match.group(1)


In [17]:
pattern = r"{<<key>>}"
input = "mydict{my_key}"
print(extract_key(input, pattern))

None


In [52]:
import re
from typing import Optional

def extract_nested_keys(string: str, pattern: str=r'\{data(\["[^"]+"\])+\}') -> Optional[list[str]]:
    match = re.search(pattern, string)
    if match:
        keys = re.findall(r'\["([^"]+)"\]', match.group(0))
        return keys

string = 'This is a test string with {data["key1"]["key2"]["key3"]["key4"]} inside.'
keys = extract_nested_keys(string)
print(keys)

['key1']


In [53]:
import re

# Example data dictionary
data = {
    "scores": {
        "accuracy": 0.95,
        "precision": 0.90,
        "recall": 0.85,
        "f1-score": 0.88
    }
}

# Your input string
input_string = """
Accuracy    : {data["scores"]["accuracy"]}
Precision   : {data["scores"]["precision"]}
Recall      : {data["scores"]["recall"]}
F1-Score    : {data["scores"]["f1-score"]}
"""

# Regular expression to match placeholders
pattern = r'\{data\["([^"]+)"\]\["([^"]+)"\]\}'

# Function to replace placeholders with values from the data dictionary
def replace_placeholders(match):
    keys = match.groups()
    value = data
    for key in keys:
        value = value.get(key, '')
    return str(value)

# Replace placeholders in the input string
output_string = re.sub(pattern, replace_placeholders, input_string)

print(output_string)


Accuracy    : 0.95
Precision   : 0.9
Recall      : 0.85
F1-Score    : 0.88

